In [3]:
import pickle
import tkinter as tk
from tkinter import ttk
import tkinter.font as tkFont
from search_individual_calculations import SemanticSearchIndividualCalculations


# Load the data from the pickle file
with open("./temporary-storage/semantic-search-object-individual-calculations.pkl", "rb") as file:
    search = pickle.load(file)

class SearchApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Semantic Search App - Precalculated")

        # Set scaling factor for widget sizes
        self.scaling_factor = 1

        # Define font size
        self.font_size = 12  # You can adjust this value to make the text bigger or smaller

        # Define font
        self.font = tkFont.Font(size=self.font_size)

        # Input field
        self.query_entry = ttk.Entry(master, width=40 * self.scaling_factor, font=self.font)
        self.query_entry.grid(row=0, column=0, padx=10 * self.scaling_factor, pady=10 * self.scaling_factor)

        # Search button
        self.search_button = ttk.Button(master, text="Search", command=self.perform_search, style='TButton')
        self.search_button.grid(row=0, column=1, padx=10 * self.scaling_factor, pady=10 * self.scaling_factor)

        # Configure style
        style = ttk.Style()
        style.configure('TButton', font=self.font)
        style.configure('TLabel', font=self.font)
        style.configure('TCheckbutton', font=self.font)

        # Input threshold slider label
        ttk.Label(master, text="Input threshold").grid(row=1, column=0, padx=10 * self.scaling_factor)

        # Search threshold slider label
        ttk.Label(master, text="Search threshold").grid(row=1, column=1, padx=10 * self.scaling_factor)

        # Number of results slider label
        ttk.Label(master, text="Number of Results").grid(row=1, column=2, padx=10 * self.scaling_factor)

        # Number of results slider
        self.results_slider = ttk.Scale(master, from_=1, to=50, orient=tk.HORIZONTAL, length=200 * self.scaling_factor)
        self.results_slider.set(3)
        self.results_slider.grid(row=2, column=2, padx=10 * self.scaling_factor)

        # Input threshold slider - without command
        self.input_threshold_slider = ttk.Scale(master, from_=0.5, to=1, orient=tk.HORIZONTAL, length=200 * self.scaling_factor)
        self.input_threshold_slider.set(0.9)
        self.input_threshold_slider.grid(row=2, column=0, padx=10 * self.scaling_factor)

        # Search threshold slider - without command
        self.search_threshold_slider = ttk.Scale(master, from_=0, to=0.5, orient=tk.HORIZONTAL, length=200 * self.scaling_factor)
        self.search_threshold_slider.set(0.1)
        self.search_threshold_slider.grid(row=2, column=1, padx=10 * self.scaling_factor)

        # Label for Input threshold slider
        self.input_threshold_label = ttk.Label(master, text=f"Input threshold: {self.input_threshold_slider.get():.2f}")
        self.input_threshold_label.grid(row=3, column=0, padx=10 * self.scaling_factor)

        # Label for Search threshold slider
        self.search_threshold_label = ttk.Label(master, text=f"Search threshold: {self.search_threshold_slider.get():.2f}")
        self.search_threshold_label.grid(row=3, column=1, padx=10 * self.scaling_factor)

        # Label for Number of Results slider
        self.results_label = ttk.Label(master, text=f"Number of Results: {int(self.results_slider.get())}")
        self.results_label.grid(row=3, column=2, padx=10 * self.scaling_factor)

        # Toggle Button State
        self.toggle_button_state = tk.BooleanVar(value=True)

        # Toggle Button
        self.toggle_button = ttk.Checkbutton(master, text="Include direct findings?", variable=self.toggle_button_state)
        self.toggle_button.grid(row=0, column=2, padx=10 * self.scaling_factor, pady=10 * self.scaling_factor)

        # Output field
        self.output_text = tk.Text(master, wrap=tk.WORD, height=25, width=50, font=self.font)  # Adjust height and width as needed
        self.output_text.grid(row=5, column=0, columnspan=3, padx=10 * self.scaling_factor, pady=10 * self.scaling_factor, sticky="ew")

        # Scrollbar
        self.scrollbar = ttk.Scrollbar(master, command=self.output_text.yview)
        self.scrollbar.grid(row=5, column=3, sticky="ns")

        self.output_text['yscrollcommand'] = self.scrollbar.set
        # Set command for both sliders after all attributes are set
        self.input_threshold_slider.config(command=self.update_sliders)
        self.search_threshold_slider.config(command=self.update_sliders)
        self.results_slider.config(command=self.update_sliders)

    def update_sliders(self, event):
        # Update the text in the labels when sliders are moved
        self.input_threshold_label["text"] = f"Input threshold: {self.input_threshold_slider.get():.2f}"
        self.search_threshold_label["text"] = f"Search threshold: {self.search_threshold_slider.get():.2f}"
        self.results_label["text"] = f"Number of Results: {int(self.results_slider.get())}"

    def perform_search(self):
        self.output_text.config(state=tk.NORMAL)
        self.output_text.delete(0.01, tk.END)
        self.output_text.insert(tk.END, "searching ...")
        self.output_text.config(state=tk.DISABLED)
        self.master.update_idletasks() 
        self.master.after(50, self.show_results)

    def show_results(self):
        result = search.make_publications_readable(search.search_for_query(self.query_entry.get(), self.input_threshold_slider.get(), self.search_threshold_slider.get(), int(self.results_slider.get()), self.toggle_button_state.get()))
        self.output_text.config(state=tk.NORMAL)
        self.output_text.delete(1.0, tk.END)
        self.output_text.insert(tk.END, result)
        self.output_text.config(state=tk.DISABLED)
        self.master.update_idletasks()
        
if __name__ == "__main__":
    root = tk.Tk()
    app = SearchApp(root)
    root.mainloop()